In [ ]:
file_name = "example_ct"

In [24]:
# Convert Vascular model database VTI files to NIFTI as required by the segmentator we are using

import os
import SimpleITK as sitk
from vmtk import pypes
from vmtk import vmtkscripts

for directory in os.listdir("./repository/"):
    print(directory)
    path = "./repository/" + directory + "/Images/";
    if os.path.isdir(path):
        for file in os.listdir(path):
            in_path = path + file
            nifti_file = file.replace(".vti", ".nii.gz")
            out_path = "./data/nifti/" + nifti_file
            args = "vmtkimagereader -ifile " + in_path + " --pipe vmtkimagewriter -ofile " + out_path
            
            print("Converted " + out_path);
            
            res = pypes.PypeRun(args);

            img = sitk.ReadImage(out_path)
            spc = img.GetSpacing()
            size = img.GetSize()
            direction = img.GetDirection()
            origin = img.GetOrigin()
            
            nda = sitk.GetArrayFromImage(img);
            d1 = spc[0] * size[0]
            d2 = spc[1] * size[1]
            d3 = spc[2] * size[2]
            
            sp1 = spc[0]
            sp2 = spc[1]
            sp3 = spc[2]

            # Check if the image dimensions make sense and don't need rescaling
            
            if (d1 < 100):
                sp1 *= 10
            if (d2 < 100):
                sp2 *= 10
            if (d3 < 100):
                sp3 *= 10
            
            img.SetSpacing([sp1, sp2, sp3]);
        
            sitk.WriteImage(img, out_path);

            print("Processed " + out_path)

0003_H_AO_SVD
Converted ./data/nifti/OSMSC0065-cm.nii.gz

Automatic piping vmtkimagereader
Parsing options vmtkimagereader
    InputFileName = ./repository/0003_H_AO_SVD/Images/OSMSC0065-cm.vti
Explicit piping vmtkimagereader
Input vmtkimagereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    UseITKIO = 1
    Image = 0
    InputFileName = ./repository/0003_H_AO_SVD/Images/OSMSC0065-cm.vti
    InputFilePrefix = 
    InputFilePattern = 
    DataExtent = [-1, -1, -1, -1, -1, -1]
    HeaderSize = 0
    DataSpacing = [1.0, 1.0, 1.0]
    DataOrigin = [0.0, 0.0, 0.0]
    DesiredOrientation = native
    DataByteOrder = littleendian
    DataScalarType = float
    FileDimensionality = 3
    Flip = [0, 0, 0]
    AutoOrientDICOMImage = 1
    ImageOutputFileName = 
Executing vmtkimagereader ...
Reading VTK XML image file.
Spacing 0.036914 0.036914 0.050000
Origin -8.766610 0.833346 -19.197500
Dimensions 512 512 296
Done executing vmtkimagereader.
Output vmtkimagereader 

Processed ./data/nifti/pre-op_mm.nii.gz
0147_H_CORO_SVGD
Converted ./data/nifti/6397SC_1.nii.gz

Automatic piping vmtkimagereader
Parsing options vmtkimagereader
    InputFileName = ./repository/0147_H_CORO_SVGD/Images/6397SC_1.vti
Explicit piping vmtkimagereader
Input vmtkimagereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    UseITKIO = 1
    Image = 0
    InputFileName = ./repository/0147_H_CORO_SVGD/Images/6397SC_1.vti
    InputFilePrefix = 
    InputFilePattern = 
    DataExtent = [-1, -1, -1, -1, -1, -1]
    HeaderSize = 0
    DataSpacing = [1.0, 1.0, 1.0]
    DataOrigin = [0.0, 0.0, 0.0]
    DesiredOrientation = native
    DataByteOrder = littleendian
    DataScalarType = float
    FileDimensionality = 3
    Flip = [0, 0, 0]
    AutoOrientDICOMImage = 1
    ImageOutputFileName = 
Executing vmtkimagereader ...
Reading VTK XML image file.
Spacing 0.041797 0.041797 0.062500
Origin -7.230000 -13.960001 -30.012500
Dimensions 512 512 370
Done executing v

Processed ./data/nifti/11908_modified_v2.nii.gz
0158_H_CORO_SVGD
Converted ./data/nifti/128301_2008.nii.gz

Automatic piping vmtkimagereader
Parsing options vmtkimagereader
    InputFileName = ./repository/0158_H_CORO_SVGD/Images/128301_2008.vti
Explicit piping vmtkimagereader
Input vmtkimagereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    UseITKIO = 1
    Image = 0
    InputFileName = ./repository/0158_H_CORO_SVGD/Images/128301_2008.vti
    InputFilePrefix = 
    InputFilePattern = 
    DataExtent = [-1, -1, -1, -1, -1, -1]
    HeaderSize = 0
    DataSpacing = [1.0, 1.0, 1.0]
    DataOrigin = [0.0, 0.0, 0.0]
    DesiredOrientation = native
    DataByteOrder = littleendian
    DataScalarType = float
    FileDimensionality = 3
    Flip = [0, 0, 0]
    AutoOrientDICOMImage = 1
    ImageOutputFileName = 
Executing vmtkimagereader ...
Reading VTK XML image file.
Spacing 0.040625 0.040625 0.125000
Origin -5.770000 -11.380000 -21.025000
Dimensions 512 512 145

In [26]:
# TotalSegmentator usage

from totalsegmentator.python_api import totalsegmentator as ts

indir = "./data/nifti/" + file_name + ".nii.gz"
outdir = "./segmentations/" + file_name + "/"

print(indir)

if not os.path.exists(outdir) :
    os.makedirs(outdir)

ts(indir, outdir)

./data/nifti/OSMSC0146-cm.nii.gz

If you use this tool please cite: https://doi.org/10.48550/arXiv.2208.05868

Resampling...
  Resampled in 5.38s
Predicting part 1 of 5 ...
Predicting part 2 of 5 ...
Predicting part 3 of 5 ...
Predicting part 4 of 5 ...
Predicting part 5 of 5 ...
  Predicted in 205.76s
Resampling...
Saving segmentations...


  0%|          | 0/104 [00:00<?, ?it/s]

  Saved in 38.03s


In [ ]:
# Process and format segmented files for visualization

indir = "./segmentations/" + file_name + "/";
outdir = "./segmentations/" + file_name + "/mhd/";

print(indir)

if not os.path.exists(outdir) :
    os.makedirs(outdir)

for file in os.listdir(indir):
    if os.path.isfile(indir + file) and ".nii.gz" in file :
        img = sitk.ReadImage(indir + file, outputPixelType = sitk.sitkInt32)

        path = outdir + file.replace(".nii.gz", ".mhd")
        mask_filter = sitk.MultiplyImageFilter()
        res = mask_filter.Execute(main, img)

        minmax_filter = sitk.MinimumMaximumImageFilter()
        minmax_filter.Execute(img)
        
        if minmax_filter.GetMaximum() != 0 :
            sitk.WriteImage(res, path)

In [27]:
# More lightweight solution for just the layer mask

indir = "./segmentations/" + file_name + "/";
outdir = "./segmentations/" + file_name + "/mhd/";

print(indir)

if not os.path.exists(outdir) :
    os.makedirs(outdir)
    
for file in os.listdir(indir):
    if os.path.isfile(indir + file) and ".nii.gz" in file :
        img = sitk.ReadImage(indir + file)
        
        path = outdir + file.replace(".nii.gz", ".mhd")
        scale_filter = sitk.ShiftScaleImageFilter()
        scale_filter.SetScale(100)
        res = scale_filter.Execute(img)

        minmax_filter = sitk.MinimumMaximumImageFilter()
        minmax_filter.Execute(img)
        
        if minmax_filter.GetMaximum() != 0 :
            sitk.WriteImage(res, path)

./segmentations/OSMSC0146-cm/


In [28]:
in_path = "./data/nifti/" + file_name + ".nii.gz"
out_path = "./data/mhd/" + file_name + ".mhd"

img = sitk.ReadImage(in_path)

nda = sitk.GetArrayFromImage(img)
print(img.GetSize())

sitk.WriteImage(img, out_path)

print(out_path)

(512, 512, 234)
./data/mhd/OSMSC0146-cm.mhd
